In [1]:
import time
import requests
import csv
from fuzzywuzzy import fuzz

# loads info for API data
url = "https://api.football-data.org/v4/"
headers = {
    "X-Auth-Token": "011d6c8e7a0b4d6493fcb8ef61b20bb2"          # api key here
}

### Leagues

In [2]:
def getInsert(league_id, league_name, year_founded, recurrence, num_teams):
    return f"INSERT INTO League(year_founded, league_name, league_id, num_teams, recurrence) VALUES ('{year_founded}', '{league_name}', '{league_id}', '{num_teams}', '{recurrence}');\n"

with open("../populate/1_leagues.sql", "w") as f:
    f.write(getInsert(
        league_id="PL",
        league_name="Premier League",
        year_founded="1992",
        recurrence="1",
        num_teams="20"
    ))
    f.write(getInsert(
        league_id="ELC",
        league_name="English League Championship",
        year_founded="2004",
        recurrence="1",
        num_teams="24"
    ))

## English Leagues

### Teams

In [3]:
teams = {}
id = 1000   # starter value
competitions = {"PL": range(2020, 2023), "ELC": range(2020, 2023)}  # stores ranges of years in whcih data is available

for competition in competitions:

    for season in competitions[competition]:

        params = {
            "season": season
        }

        skip = False    # determines if the information cannot be accessed

        while True:
            try:
                data = requests.get(f"{url}competitions/{competition}/teams", headers=headers, params=params).json()
                _ = data["teams"]   # checks if the data is available
                break
            except: 
                if data["errorCode"] == 429:    # rate limited
                    print("Rate limited. Sleeping for 60 seconds.")
                    time.sleep(60)
                elif data["errorCode"] == 403:  # no permissions
                    print(f"Unable to retrieve {season} data for {competition}. Skipping.")
                    skip = True
                    break

        if skip: continue

        for team in data["teams"]:  # stores information about the team
            teams[team["name"]] = {"id": -1, "founding": team["founded"], "shortname": team["shortName"]}

for team in teams:
    id += 1             # incrementally assigns id-numbers to each individual team
    teams[team]["id"] = id

# gets the sql query for inserting informaion about the team
def getInsert(team_name, team_id, founded):
    output = f"INSERT INTO Team(team_name, team_id, year_founded, country) VALUES ('{team_name}', '{team_id}', '{founded}', 'Great Britain');\n"
    return output.replace("'None'", "NULL")

with open("../populate/2_englishteams.sql", "w") as f:  # write to file
    for team, info in teams.items():
        f.write(getInsert(team_name=team, team_id=info["id"], founded=info["founding"]))


In [4]:
team_ids = {}
for name, info in teams.items():      # for usage in the code elsewhere, such as playerteam
    team_ids[name] = info['id']
    team_ids[info['shortname']] = info['id']
# print(team_ids)

### Players

In [5]:
id = 1000   # starter value

def getInsertPlayer(name, id, position=None):

    # determines how to split the player's name
    name = name.replace("'", "").split()
    middle_initial = None
    if len(name) == 1:
        first_name, last_name = None, name[0]   # only one name given - "Sokratis"
    elif len(name) == 2:
        first_name, last_name = name            # both names given 
    elif len(name) == 3:
        first_name, middle_initial, last_name = name[0], name[1][0].upper(), name[2]    # Ahmed El Mohamady - three names given
    else:   
        first_name, last_name = name[0], name[-1]

    # truncates name
    if first_name != None and len(first_name) > 10:
        first_name = first_name[:10]
    if len(last_name) > 20:
        last_name = last_name[:20]

    output = f"INSERT INTO Player(player_id, first_name, last_name, middle_initial, field_position) VALUES('{id}', '{first_name}', '{last_name}', '{middle_initial}', '{position}'); \n"
    return output.replace("'None'", "NULL")

players = {}

with open("../../data/pl-players-2020.csv", "r") as f_read:
    with open("../populate/3_englishplayers.sql", "w") as f_write:
        reader = csv.DictReader(f_read)
        for player in reader:
            id += 1     # increments id to make a new one
            f_write.write(getInsertPlayer(
                name=player["Name"],
                position=player["Position"],
                id=id
            ))
            players[player["Name"]] = id

### PlayerTeam

In [6]:
year = 2020

# sql insert statement for player-tean relations
def getInsertPlayerTeam(id, player, team, earnings=None, date=None):
    output = f"INSERT INTO PlayerTeam(playerteam_id, player_id, team_id, earnings, date_joined) VALUES('{id}', '{player}', '{team}', '{earnings}' ,'{date}');\n"
    return output.replace("'None'", "NULL")

def getTeamID(teams, team):
    for comp_team in teams:
        # check if the team name is essentailly similar
        if fuzz.ratio(comp_team.replace("FC", "").strip(), team.replace("-", " ").strip()) > 90:
            return teams[comp_team]     # returns id if possible
    return None

id = 1000
with open(f"../../data/pl-players-{year}.csv", "r") as f_read:
    with open("../populate/4_englishplayerteams.sql", "w") as f_write:
        reader = csv.DictReader(f_read)
        for player in reader:
            id += 1
            f_write.write(getInsertPlayerTeam(
                player=players[player["Name"]],
                team=getTeamID(team_ids, player["Club"]),
                id=id,
                date=f"{year}-08-01"    # no date given - assumed it was sometime in august 2020 because dataset was for 2020 
            ))

### PlayerStat

In [7]:
def getInsertPlayerStat(id, games, goals, assists, player, hat_tricks=None):
    output = f"""INSERT INTO PlayerStat(stat_id, player_id, games_played, goals_scored, hat_tricks, assists) 
                 VALUES('{id}', '{player}', '{games}', '{goals}', '{hat_tricks}', '{assists}'); \n"""
    return output.replace("'None'", "NULL")

id = 1000
with open("""TODO""", "r") as f_read:
    with open("../populate/5_englishplayerstats.sql", "w") as f_write:
        reader = csv.DictReader(f_read)
        for player in reader:
            id += 1
            f_write.write(getInsertPlayerStat(
                id=id,
                player=players[player["Name"]],
                goals=player["Goals"],
                assists=player["Assists"],
                gamse=player[""]
            ))

FileNotFoundError: [Errno 2] No such file or directory: 'TODO'